# 0.0 Necessery tasks before work

In [3]:
# Import the libraries that are needed
import pandas as pd, numpy as np

In [4]:
# Loading Data
df = pd.read_csv('cafe_sales_dirty.csv')

# 1.0 Overall Checking

In [5]:
# Observing the first 10 records
df.head(10)

,OrderID,Date,CustomerName,Item,Quantity,Price,PaymentType,Notes
0,1,NaN,UNKNOWN,NaN,NaN,NaN,Cash,Extra sugar
1,2,2025/12/04,Eve,Tea,1,NaN,Online,No ice
2,"3,""Dec 3, 2025"",UNKNOWN,Cake,Two,3.50,Cash,Spi...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,2025/12/04,UNKNOWN,Tea,NaN,5.00,NaN,Extra sugar
4,5,2025/12/04,NaN,ERROR,NaN,NaN,Online,NaN
5,6,12/02/2025,Unknown,Juice,Two,NaN,Online,UNKNOWN
6,"7,""Dec 3, 2025"",David,Juice,,,Card,",NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,"8,""Dec 3, 2025"",UNKNOWN,Cake,,N/A,Card,",NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,"9,""Dec 3, 2025"",Charlie,,,3.50,Cash,Spilled order",NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,10,2025/12/04,UNKNOWN,Tea,ERROR,3.50,Online,Extra sugar


In [6]:
# Observing the last 10 records
df.tail(10)

,OrderID,Date,CustomerName,Item,Quantity,Price,PaymentType,Notes
990,991,12/02/2025,Charlie,Juice,3,5.00,Online,No ice
991,992,12/02/2025,NaN,Coffee,3,NaN,NaN,Extra sugar
992,"993,""Dec 3, 2025"",,Cake,,,UNKNOWN,Extra sugar",NaN,NaN,NaN,NaN,NaN,NaN,NaN
993,"994,""Dec 3, 2025"",Bob,ERROR,,ERROR,Card,",NaN,NaN,NaN,NaN,NaN,NaN,NaN
994,995,2025/12/04,Bob,ERROR,1,3.50,UNKNOWN,Spilled order
995,996,UNKNOWN,Unknown,Sandwich,3,NaN,NaN,Spilled order
996,997,2025/12/04,Charlie,NaN,2,5.00,Online,UNKNOWN
997,998,2025-12-01,Eve,Cake,3,5.00,NaN,Spilled order
998,999,2025-12-01,UNKNOWN,Coffee,1,NaN,NaN,Extra sugar
999,1000,2025/12/04,NaN,Cake,ERROR,3.50,UNKNOWN,UNKNOWN


In [7]:
# Observing some stats of all columns
df.describe(include='all')

,OrderID,Date,CustomerName,Item,Quantity,Price,PaymentType,Notes
count,1000,672,743,734,691,556,677,578
unique,1000,4,7,6,5,4,4,4
top,1,2025-12-01,Eve,Tea,Two,3.50,Online,No ice
freq,1,181,116,133,147,150,181,151


In [8]:
# Observe non-nulls for each column
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   OrderID       1000 non-null   object
 1   Date          672 non-null    object
 2   CustomerName  743 non-null    object
 3   Item          734 non-null    object
 4   Quantity      691 non-null    object
 5   Price         556 non-null    object
 6   PaymentType   677 non-null    object
 7   Notes         578 non-null    object
dtypes: object(8)
memory usage: 62.6+ KB


# 2.0 Creat a complete info table for columns

In [9]:
# Creat a table that includes some infos for each column
info_df = pd.DataFrame(
    {
        'columns': df.columns,
        'dtype' : df.dtypes,
        'Non-Null': df.notnull().sum(),
        'Null': df.isna().sum(),
        'Null_Percentage': ((df.isna().sum())/len(df) * 100).round(2),
        'Unique': df.nunique(),
        'Samples': df.sample(5).T.values.tolist()
    }
)

info_df

,columns,dtype,Non-Null,Null,Null_Percentage,Unique,Samples
OrderID,OrderID,object,1000,0,0.0,1000,"[580, 298, 745, 795, 630]"
Date,Date,object,672,328,32.8,4,"[nan, nan, 2025-12-01, 2025-12-01, 12/02/2025]"
CustomerName,CustomerName,object,743,257,25.7,7,"[Alice, nan, Unknown, nan, Charlie]"
Item,Item,object,734,266,26.6,6,"[Coffee, Coffee, Coffee, Sandwich, Cake]"
Quantity,Quantity,object,691,309,30.9,5,"[ERROR, Two, ERROR, 1, Two]"
Price,Price,object,556,444,44.4,4,"[ERROR, nan, nan, 5.00, nan]"
PaymentType,PaymentType,object,677,323,32.3,4,"[Cash, Cash, nan, UNKNOWN, Card]"
Notes,Notes,object,578,422,42.2,4,"[UNKNOWN, UNKNOWN, Extra sugar, UNKNOWN, nan]"


# 3.0 Cleaning! 
yay:)

In [10]:
# Making a copy of Data; for preventing from data loss
cleaned_df = df.copy()

## 3.1 Clean Order Id

In [11]:
def OrderID_standardize(record):
    """It standardize OrderID column"""
    if isinstance(record, int): # If the record is an integer: Just return it self
        return record 
    elif isinstance(record, str): # If the record is a string
        try: 
            return int(record) # If it's a integer in string
        
        except: # If all of the record is just in the Id
            tmp_str = record.split(',')
            tmp_str[1] = (tmp_str[1]+tmp_str[2]).strip('"')
            tmp_str.pop(2)
            cleaned_df[cleaned_df.OrderID == record] = tmp_str 

            return int(tmp_str[0]) # Prventing from OrderID=None
    else:
        return np.nan

In [12]:
# Apply the function for the first column
cleaned_df['OrderID'] = cleaned_df['OrderID'].apply(OrderID_standardize)

### Changes

In [14]:
# Seeing the difference between original OrderID with cleaned OrderID

OrderID_changes = (df['OrderID'] != cleaned_df['OrderID']).sum()
print(f"This step's changing was {OrderID_changes}!")

This step's changing was 1000!


## 3.2 Clean Date

In [15]:
# The unique values of Date column and their quantity(doesn't show the np.nan)
cleaned_df['Date'].value_counts()

Date
2025-12-01    181
2025/12/04    172
12/02/2025    164
UNKNOWN       155
Dec 3 2025    154
Name: count, dtype: int64

In [ ]:
def Date_standardize(date):
    """It makes the date column standard"""
    
    if date == 'UNKNOWN':
        return np.nan
    elif isinstance(date, str):
        date = pd.to_datetime(date).date()
        return date
    else:
        return np.nan

In [17]:
# Apply the function to second column for cleaning it

cleaned_df['Date'] = cleaned_df['Date'].apply(Date_standardize)

In [18]:
# Convert the date column to show that column's type is datetime
cleaned_df['Date'] = pd.to_datetime(cleaned_df['Date'])

### Changes

In this step we will just see the value counts after cleaning, because we converted the whole column to date type, it would show us 1000/1000 changes!

In [19]:
cleaned_df.Date.value_counts()

Date
2025-12-01    181
2025-12-04    172
2025-12-02    164
2025-12-03    154
Name: count, dtype: int64

## 3.3 Clean CustomerName

In [20]:
# The unique values of Date column and their quantity(doesn't show the np.nan)

cleaned_df.CustomerName.value_counts()

CustomerName
Charlie    136
Eve        133
Alice      131
UNKNOWN    129
Bob        124
Unknown    117
David      105
            22
Name: count, dtype: int64

In [21]:
def CustomerName_standardize(name):
    """It makes the CustomerName column standard"""

    if name in ['', 'UNKNOWN', 'Unknown']:
        return np.nan
    elif not(pd.isna(name)):
        return name.title()
    else:
        return np.nan

In [22]:
# Applies the function to the CustomerName column to make it clean

cleaned_df['CustomerName'] = cleaned_df['CustomerName'].apply(CustomerName_standardize)
cleaned_df['CustomerName']

0          NaN
1          Eve
2          NaN
3          NaN
4          NaN
        ...   
995        NaN
996    Charlie
997        Eve
998        NaN
999        NaN
Name: CustomerName, Length: 1000, dtype: object

### Changes

In [23]:
# The unique values of Date column and their quantity(doesn't show the np.nan)

cleaned_df['CustomerName'].value_counts()

CustomerName
Charlie    136
Eve        133
Alice      131
Bob        124
David      105
Name: count, dtype: int64

In [24]:
# Seeing the difference between original CustomerName with cleaned CustomerName

CustomerName_changes = (cleaned_df['CustomerName']!=df['CustomerName']).sum()
f"This step's changes are {CustomerName_changes}"

"This step's changes are 465"

## 3.4 Clean Item

In [25]:
# The unique values of Date column and their quantity(doesn't show the np.nan)

cleaned_df['Item'].value_counts()

Item
Cake        161
Tea         152
ERROR       150
Juice       139
Coffee      139
Sandwich    122
             25
Name: count, dtype: int64

In [26]:
def Item_standardized(item):
    """It makes the Item column standard"""

    if item in ['ERROR', '', np.nan]:
        return np.nan
    else:
        return item

In [27]:
# Applies the function to the Item column to make it clean

cleaned_df['Item'] = cleaned_df['Item'].apply(Item_standardized)
cleaned_df['Item']

0           NaN
1           Tea
2          Cake
3           Tea
4           NaN
         ...   
995    Sandwich
996         NaN
997        Cake
998      Coffee
999        Cake
Name: Item, Length: 1000, dtype: object

### Changes

In [28]:
# Seeing the difference between original Item with cleaned Item

Item_changes = (cleaned_df['Item'] != df['Item']).sum()
f"This step's changing was {Item_changes}"

"This step's changing was 397"

## Making a temporary dataframe

In [29]:
# Getting a temporary version of my df, because I'm extremely excited:)))
cleaned_df_csv = cleaned_df.to_csv('tempcleaneddf')

In [30]:
# Read the csv I made

pd.read_csv('tempcleaneddf')

,Unnamed: 0,OrderID,Date,CustomerName,Item,Quantity,Price,PaymentType,Notes
0,0,1,NaN,NaN,NaN,NaN,NaN,Cash,Extra sugar
1,1,2,2025-12-04,Eve,Tea,1,NaN,Online,No ice
2,2,3,2025-12-03,NaN,Cake,Two,3.50,Cash,Spilled order
3,3,4,2025-12-04,NaN,Tea,NaN,5.00,NaN,Extra sugar
4,4,5,2025-12-04,NaN,NaN,NaN,NaN,Online,NaN
...,...,...,...,...,...,...,...,...,...
995,995,996,NaN,NaN,Sandwich,3,NaN,NaN,Spilled order
996,996,997,2025-12-04,Charlie,NaN,2,5.00,Online,UNKNOWN
997,997,998,2025-12-01,Eve,Cake,3,5.00,NaN,Spilled order
998,998,999,2025-12-01,NaN,Coffee,1,NaN,NaN,Extra sugar


## 3.5 Clean Quantity

In [31]:
# The unique values of quantity column and their quantity(doesn't show the np.nan)

cleaned_df['Quantity'].value_counts()

Quantity
Two      171
1        170
ERROR    170
3        158
2        147
          29
Name: count, dtype: int64

In [32]:
def Quantity_standardize(quantity):
    """It makes the Quantity column standard"""

    Title_numbers = ['One', 'Two', 'Three', 'Four', 'Five', 'Six', 'Seven', 'Eight', 'Nine', 'Ten']
    if quantity in ['', 'ERROR',np.nan]:
        return np.nan
    elif quantity in Title_numbers:
        return Title_numbers.index(quantity) + 1 
    elif isinstance(quantity, str):
        return int(quantity)
    else:
        return np.nan

In [33]:
# Applies the function to the quantity column to make it clean and convert the type into Int64(holdes both np.nan and integers)

cleaned_df['Quantity'] = cleaned_df['Quantity'].apply(Quantity_standardize).astype('Int64')

### Changes

In [34]:
# Seeing the difference between original Quantity with cleaned Quantity

quantity_changes= (cleaned_df['Quantity']!=df['Quantity']).sum()
f"This step's changes was {quantity_changes}"

"This step's changes was 547"

## 3.6 Clean Price

In [35]:
# The unique values of Date column and their quantity(doesn't show the np.nan)

cleaned_df['Price'].value_counts()

Price
3.50     175
2.75     172
ERROR    154
5.00     152
          30
N/A       27
Name: count, dtype: int64

In [36]:
def price_standardize(price):
    """It makes the Price column standard"""
    
    if price in ['', 'ERROR', 'N/A', np.nan]:
        return np.nan
    elif isinstance(price, str):
        return price
    else:
        return np.nan

In [37]:
# Applies the function to the price column to make it clean

cleaned_df['Price'] = cleaned_df['Price'].apply(price_standardize)
cleaned_df['Price']

0       NaN
1       NaN
2      3.50
3      5.00
4       NaN
       ... 
995     NaN
996    5.00
997    5.00
998     NaN
999    3.50
Name: Price, Length: 1000, dtype: object

### Changes

In [38]:
# Seeing the difference between original price with cleaned price

price_changes = (cleaned_df['Price']!=df['Price']).sum()
f"This step's changes was {price_changes}"

"This step's changes was 577"

## 3.7 Clean PaymentType

In [39]:
# The unique values of Date column and their quantity(doesn't show the np.nan)

cleaned_df['PaymentType'].value_counts()

PaymentType
Online     211
UNKNOWN    205
Cash       200
Card       194
            21
Name: count, dtype: int64

In [40]:
def PaymentType_standardize(payment):
    """It makes the PaymentType column standard"""
    
    if payment in ['UNKNOWN', '', np.nan]:
        return np.nan
    elif isinstance(payment, str):
        return payment
    else:
        return np.nan

In [41]:
# Applies the function to the payment column to make it clean

cleaned_df['PaymentType'] = cleaned_df['PaymentType'].apply(PaymentType_standardize)
cleaned_df['PaymentType']

0        Cash
1      Online
2        Cash
3         NaN
4      Online
        ...  
995       NaN
996    Online
997       NaN
998       NaN
999       NaN
Name: PaymentType, Length: 1000, dtype: object

In [42]:
# Seeing the difference between original payment with cleaned payment

payment_changes = (cleaned_df['PaymentType'] != df['PaymentType']).sum()
f"This step's changes was {payment_changes}"

"This step's changes was 495"

## 3.8 Clean Notes

In [43]:
# The unique values of Date column and their quantity(doesn't show the np.nan)

cleaned_df['Notes'].value_counts()

Notes
No ice           176
UNKNOWN          169
Spilled order    166
Extra sugar      165
                  56
Name: count, dtype: int64

In [44]:
def Notes_standardize(note):
    """It makes the Notes column standard"""
    
    if note in ['', 'UNKNOWN',np.nan, 'Spilled order']:
        return np.nan
    elif isinstance(note, str):
        return note
    else:
        np.nan

In [45]:
# Applies the function to the CustomerName column to make it clean

cleaned_df['Notes'] = cleaned_df['Notes'].apply(Notes_standardize)
cleaned_df['Notes']

0      Extra sugar
1           No ice
2              NaN
3      Extra sugar
4              NaN
          ...     
995            NaN
996            NaN
997            NaN
998    Extra sugar
999            NaN
Name: Notes, Length: 1000, dtype: object

In [46]:
# Seeing the difference between original notes with cleaned notes

Notes_changes = (cleaned_df['Notes']!= df['Notes']).sum()
f"This step's changes was {Notes_changes}"

"This step's changes was 705"

# 4.0 Final fixes

In [47]:
# Dropping the rows that have at list 4 non-null values

cleaned_df.dropna(thresh=4, inplace=True)
cleaned_df

,OrderID,Date,CustomerName,Item,Quantity,Price,PaymentType,Notes
1,2,2025-12-04,Eve,Tea,1,NaN,Online,No ice
2,3,2025-12-03,NaN,Cake,2,3.50,Cash,NaN
3,4,2025-12-04,NaN,Tea,<NA>,5.00,NaN,Extra sugar
5,6,2025-12-02,NaN,Juice,2,NaN,Online,NaN
6,7,2025-12-03,David,Juice,<NA>,NaN,Card,NaN
...,...,...,...,...,...,...,...,...
994,995,2025-12-04,Bob,NaN,1,3.50,NaN,NaN
996,997,2025-12-04,Charlie,NaN,2,5.00,Online,NaN
997,998,2025-12-01,Eve,Cake,3,5.00,NaN,NaN
998,999,2025-12-01,NaN,Coffee,1,NaN,NaN,Extra sugar


# 5.0 Make a new csv of cleaned data

In [48]:
cleaned_df.to_parquet('FinalCleanedData.parquet', index=False)

# 6.0 Taking a look at the final result

In [49]:
pd.read_parquet('FinalCleanedData.parquet')

,OrderID,Date,CustomerName,Item,Quantity,Price,PaymentType,Notes
0,2,2025-12-04,Eve,Tea,1,None,Online,No ice
1,3,2025-12-03,None,Cake,2,3.50,Cash,None
2,4,2025-12-04,None,Tea,<NA>,5.00,None,Extra sugar
3,6,2025-12-02,None,Juice,2,None,Online,None
4,7,2025-12-03,David,Juice,<NA>,None,Card,None
...,...,...,...,...,...,...,...,...
883,995,2025-12-04,Bob,None,1,3.50,None,None
884,997,2025-12-04,Charlie,None,2,5.00,Online,None
885,998,2025-12-01,Eve,Cake,3,5.00,None,None
886,999,2025-12-01,None,Coffee,1,None,None,Extra sugar
